<a href="https://colab.research.google.com/github/Natural-State/agol-data-workflows/blob/master/code/Colab%20notebooks/07_MOD44B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extract percentage tree, non-tree veg, and bare ground measures from MODIS VCF platform

## Import gee and authenticate

In [ ]:
import ee

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

## Input arguments for data extraction

In [ ]:
# Area of interest
aoi = ee.FeatureCollection("projects/ns-agol-rs-data/assets/LLBN")
aoi_name = "LLBN"

# GEE layer ID
layer_name_tc = "RS_014" # Percentage tree cover
layer_name_ntc = "RS_015" # Percentage non-tree vegetation cover
layer_name_nvc = "RS_016" # Percentage non-vegetated cover

# Date parameters
start_year = 2012
end_year = 2022

# Range doesn't include the stop value
year_list = ee.List(list(range(start_year, end_year+1)))

## MOD44B.006 Terra Vegetation Continuous Fields Yearly Global 250m



In [ ]:
modis = ee.ImageCollection("MODIS/006/MOD44B")

## VCF processing

In [ ]:
years = ee.List.sequence(start_year, end_year)

# Band names
# Percent_Tree_Cover
# Percent_NonTree_Vegetation
# Percent_NonVegetated

def yearly_cover(focal_year, metric):
  start = ee.Date.fromYMD(focal_year, 1, 1)
  end = ee.Date.fromYMD(focal_year, 12, 31)
  date_range = ee.DateRange(start, end)
  return modis \
        .filterDate(date_range) \
        .select(metric) \
        .mean() \
        .set({"name": focal_year, "metric": metric}) \
        .clip(aoi)

annual_tc = years.map(lambda focal_year: yearly_cover(focal_year, metric = "Percent_Tree_Cover"))
print(annual_tc.getInfo())

annual_ntc = years.map(lambda focal_year: yearly_cover(focal_year, metric = "Percent_NonTree_Vegetation"))
print(annual_ntc.getInfo())

annual_nvc = years.map(lambda focal_year: yearly_cover(focal_year, metric = "Percent_NonVegetated"))
print(annual_nvc.getInfo())

## Check an element of list
annual  = ee.Image(annual_tc.get(0))
label1 = ee.String(annual.get('name')).getInfo()
label2 = ee.String(annual.get('metric')).getInfo()
print(str(label1) + label2)
print(annual.getInfo())


In [ ]:
ee.List.length(annual_tc).getInfo()

## Mapping images

In [ ]:
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

 # Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 50,
  'palette': ['#FF0000', '#FF6600', '#FFCC00', '#99FF66', '#00FF00']
  }

# Create a folium map object.
my_map = folium.Map(location=[0.1591, 37.3729], zoom_start=12)

# Add layers

annual_image = annual_tc
# annual_image = annual_ntc
# annual_image = annual_nvc

# for i in range(ee.List.length(annual_image).getInfo()):
for i in range(5):
  print(i)
  year_mosaic = ee.Image(annual_image.get(i))
  label = ee.String(year_mosaic.get('name')).getInfo()
  my_map.add_ee_layer(year_mosaic, vis_params, label)

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

## Export data - create task

`filenamePrefix` should be in format: place_layer_timeperiod

In [ ]:
def export_vcf_images(annual_img, layer_name):
  for i in  range(ee.List.length(annual_img).getInfo()):
    output_img =  ee.Image(annual_img.get(i))
    output_name = f"{layer_name}_{aoi_name}_{ee.String(output_img.get('name')).getInfo()}"
    task = ee.batch.Export.image.toDrive(image = output_img,
                                     region = aoi.geometry().bounds(),
                                     description = "EXPORT IMAGE TO DRIVE",
                                     folder = "GEE_exports",
                                     fileNamePrefix = output_name,
                                     scale = 250,
                                     maxPixels = 10e12,
                                     crs = "EPSG:4326"
                                     )
    task.start()
    print("STARTED TASK ", i+1)

export_vcf_images(annual_tc, "RS_014")  # Percentage tree cover
export_vcf_images(annual_ntc, "RS_015") # Percentage non-tree vegetation cover
export_vcf_images(annual_nvc, "RS_016") # Percentage non-vegetated cover



## Check task status

[List](https://developers.google.com/earth-engine/guides/processing_environments#list-of-task-states) of task status messages (state field)


In [ ]:
tasks = ee.batch.Task.list()
for task in tasks[0:ee.List.length(year_list).getInfo()]:
  task_id = task.status()['id']
  task_state = task.status()['state']
  print(task_id, task_state)